In [1]:
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
client = OpenAI()

In [2]:
import os
import sys
import base64
import json
import pandas as pd

In [3]:
root_directory = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(os.path.join(root_directory,"src"))
doc_folder = os.getcwd()+"/../../data/"

import gptocr
import timetabledata

In [15]:
import importlib
importlib.reload(gptocr)
importlib.reload(timetabledata)

<module 'gptocr' from 'c:\\Users\\kkoaz\\Documents\\projects\\product\\timetable-detect\\timetable-detect\\src\\gptocr.py'>

# 作成した関数を活用

In [18]:
#対バンOCR
filename = "20231226_TOKYO-GIRLS-GIRLS.jpeg"
prompt_user = "この画像のタイムテーブルをデータ化して"
return_json = gptocr.getocr_taiban_filename(filename, prompt_user)
return_json

{'ライブ名': 'TOKYO GIRLS GIRLS',
 '会場名': '品川インターシティホール',
 '日付': '2023-12-26',
 'タイムテーブル': [{'グループ名': '8月のエウラルベ',
   'ライブステージ': [{'from': '8:20', 'to': '8:30'}],
   '特典会': [{'from': '8:40', 'to': '9:40', 'ブース': 'A'}]},
  {'グループ名': 'ポンコツコンポ',
   'ライブステージ': [{'from': '8:30', 'to': '8:40'}],
   '特典会': [{'from': '8:50', 'to': '9:50', 'ブース': 'B'}]},
  {'グループ名': 'MiX BeRRY',
   'ライブステージ': [{'from': '8:40', 'to': '8:50'}],
   '特典会': [{'from': '9:00', 'to': '10:00', 'ブース': 'C'}]},
  {'グループ名': 'make mie',
   'ライブステージ': [{'from': '8:50', 'to': '9:00'}],
   '特典会': [{'from': '9:10', 'to': '10:10', 'ブース': 'D'}]},
  {'グループ名': 'RINCENT#',
   'ライブステージ': [{'from': '9:00', 'to': '9:10'}],
   '特典会': [{'from': '9:20', 'to': '10:20', 'ブース': 'E'}]},
  {'グループ名': '天使にはなれない',
   'ライブステージ': [{'from': '9:10', 'to': '9:20'}],
   '特典会': [{'from': '9:30', 'to': '10:30', 'ブース': 'F'}]},
  {'グループ名': 'アルテミスの翼',
   'ライブステージ': [{'from': '9:20', 'to': '9:35'}],
   '特典会': [{'from': '9:45', 'to': '10:45', 'ブース': 'A'}]},
  {'グルー

In [94]:
#読み取ったJSONの保存
json_path = os.path.join("../../data/output/timetable_taiban/{file_name}.json".format(file_name=filename.split(".")[0]))
with open(json_path,"w",encoding = "utf8") as f:
    json.dump(return_json, f, indent = 4, ensure_ascii = False)


#保存したJSONの読み取り
json_path = "../../data/output/timetable_taiban/{file_name}.json".format(file_name=filename.split(".")[0])
with open(json_path,"r",encoding = "utf8") as f:
    return_json = json.load(f)
return_json

{'ライブ名': 'TOKYO GIRLS GIRLS',
 '会場名': '品川インターシティホール',
 '日付': '2023-12-26',
 'タイムテーブル': [{'グループ名': '8月のエウテルペ',
   'ライブステージ': {'from': '8:20', 'to': '8:30'},
   '特典会': [{'from': '8:40', 'to': '9:40', 'ブース': 'A'}]},
  {'グループ名': 'ポンコツコンポ',
   'ライブステージ': {'from': '8:30', 'to': '8:40'},
   '特典会': [{'from': '8:50', 'to': '9:50', 'ブース': 'B'}]},
  {'グループ名': 'MiX BeRRY',
   'ライブステージ': {'from': '8:40', 'to': '8:50'},
   '特典会': [{'from': '9:00', 'to': '10:00', 'ブース': 'C'}]},
  {'グループ名': 'make mie',
   'ライブステージ': {'from': '8:50', 'to': '9:00'},
   '特典会': [{'from': '9:10', 'to': '10:10', 'ブース': 'D'}]},
  {'グループ名': 'RiNCENT＃',
   'ライブステージ': {'from': '9:00', 'to': '9:10'},
   '特典会': [{'from': '9:20', 'to': '10:20', 'ブース': 'E'}]},
  {'グループ名': '天使にはなれない',
   'ライブステージ': {'from': '9:10', 'to': '9:20'},
   '特典会': [{'from': '9:30', 'to': '10:30', 'ブース': 'F'}]},
  {'グループ名': 'アルテミスの翼',
   'ライブステージ': {'from': '9:20', 'to': '9:35'},
   '特典会': [{'from': '9:45', 'to': '10:45', 'ブース': 'A'}]},
  {'グループ名': 'NiLUNLOC

In [97]:
#アイドルグループ名の修正
df_timetable = timetabledata.json_to_df(return_json)
for i,row in df_timetable.iterrows():
    group_name_correct = timetabledata.get_name_list(row["グループ名"])
    if not group_name_correct[0]:
        df_timetable.at[i,"グループ名"] = group_name_correct[1]
df_timetable

,グループ名,ライブ_from,ライブ_to,ライブ_長さ(分),特典会_from,特典会_to,ブース
0,8月のエウテルペ,08:20,08:30,10,8:40,9:40,A
1,ポンコツコンポ,08:30,08:40,10,8:50,9:50,B
2,MiX BeRRY,08:40,08:50,10,9:00,10:00,C
3,make mie,08:50,09:00,10,9:10,10:10,D
4,RiNCENT♯,09:00,09:10,10,9:20,10:20,E
5,天使にはなれない,09:10,09:20,10,9:30,10:30,F
6,アルテミスの翼,09:20,09:35,15,9:45,10:45,A
7,NiLUNLOCK,09:40,09:55,15,10:05,11:05,B
8,ZUTTOMOTTO,09:55,10:10,15,10:20,11:20,C
9,alma,10:10,10:25,15,10:35,11:35,D


# OCR

In [28]:
system_prompt_taiban = """あなたは日本語に長けていて、また日本のアイドルシーンに詳しい専門家です。
与えられる画像は日本のアイドルの対バンライブについてのタイムテーブルです。
そこから情報を読み取り、
・ライブの名前
・会場の名前
・日付
・タイムテーブル（出演するアイドルの一覧とその出演時間）
をJSON形式で出力してください。

#ライブステージと特典会について
各アイドルは「ライブステージ」と「特典会」の2種類を行う可能性があります。
ライブステージの時間と合わせて、特典会の時間も出力してください。
ユーザーから「特典会無し」と指示された場合のみ、特典会の情報は出力せずライブステージの時間のみを出力してください。

#ライブステージについて
ライブステージは一つのアイドルグループが複数回出演する場合があります。
属性として「開始時間(from)」「終了時間(to)」を持ちます。

#特典会について
特典会は一つのアイドルグループが複数回行う場合があります。
属性として「開始時間(from)」「終了時間(to)」「ブース」を持ちます。
##特典会のブースについて
ブースとは、特典会の実施場所のことで、数字やアルファベット、日本語等で示されている場合があります。
時間の脇にアルファベット1文字が記載されている場合はそれをブースとして採用してください。
特に表記が見当たらない場合には、ブースは「なし」としてください。
##終演後特典会について
特典会は「終演後物販」「終演後特典会」といった形で、ライブ終了後に複数のグループが同時に行う場合があります。
特典会の時刻が画像からはわかりにくいグループは、一律の終演後特典会で行われる場合が多いです。
終演後特典会は時刻が明記されていない場合がありますが、
その場合は最後のライブステージの終了時刻を特典会の開始時刻として、そこから60分で計算してください。
時刻が明記されている場合は、終演後特典会であっても時間を60分ではなく明記されている時間を採用してください。

#ライブステージと特典会の判別方法
ライブステージの時間より、特典会の時間のほうが長い傾向にあります。
ライブステージはだいたい20分～30分程度であることが多く、短い場合は10分、長い場合でも40分に収まることが多いです。
特典会は60分前後の場合が多いです。

#出力形式
以下の様なJSON形式で出力してください。
{
    "ライブ名":"TOKYO GIRLS GIRLS"
    ,"会場名":"ZeppDiverCity"
    ,"日付":"2024-08-03"
    ,"タイムテーブル":
    [
        {
            "グループ名":"アイドルグループA"
            ,"ライブステージ":[
            {
                "from":"10:00"
                ,"to":"10:20"
            },
            ]
            ,"特典会":[
            {
                "from":"10:30"
                ,"to":"11:30"
                ,"ブース":"A"
            },
            ]
        },
        {
            "グループ名":"架空のアイドル"
            ,"ライブステージ":[
            {
                "from":"10:25"
                ,"to":"10:45"
            },
            ]
            ,"特典会":[
            {
                "from":"10:55"
                ,"to":"11:55"
                ,"ブース":"B"
            },
            ]
        }
    ]
}
"""

In [29]:
#フェスの場合のJSONの持ち方を変える？アイドルごとではなくステージごとにするとか
system_prompt_fes = """あなたは日本語に長けていて、また日本のアイドルシーンに詳しい専門家です。
与えられる画像は日本のアイドルフェスについてのタイムテーブルです。
そこから情報を読み取り、
・ライブの名前
・会場の名前
・日付
・タイムテーブル（出演するアイドルの一覧とその出演時間）
をJSON形式で出力してください。

#ライブステージと特典会について
各アイドルは「ライブステージ」と「特典会」の2種類を行う可能性があります。
ライブステージの時間と合わせて、特典会の時間も出力してください。
ユーザーから「特典会無し」と指示された場合のみ、特典会の情報は出力せずライブステージの時間のみを出力してください。

#ライブステージについて
ライブステージは一つのアイドルグループが複数回出演する場合があります。
ステージの種類が複数あり、それらが同時並行で進行する場合があります。
複数のステージがある場合、それぞれのステージには名前がついていることが一般的です。
属性として「開始時間(from)」「終了時間(to)」「ステージ名」を持ちます。

#特典会について
特典会は一つのアイドルグループが複数回行う場合があります。
属性として「開始時間(from)」「終了時間(to)」「ブース」を持ちます。
##特典会のブースについて
ブースとは、特典会の実施場所のことで、数字やアルファベット、日本語等で示されている場合があります。
時間の脇にアルファベット1文字が記載されている場合はそれをブースとして採用してください。
特に表記が見当たらない場合には、ブースは「なし」としてください。
##終演後特典会について
特典会は「終演後物販」「終演後特典会」といった形で、ライブ終了後に複数のグループが同時に行う場合があります。
特典会の時刻が画像からはわかりにくいグループは、一律の終演後特典会で行われる場合が多いです。
終演後特典会は時刻が明記されていない場合がありますが、
その場合は最後のライブステージの終了時刻を特典会の開始時刻として、そこから60分で計算してください。
時刻が明記されている場合は、終演後特典会であっても時間を60分ではなく明記されている時間を採用してください。

#ライブステージと特典会の判別方法
ライブステージの時間より、特典会の時間のほうが長い傾向にあります。
ライブステージはだいたい20分～30分程度であることが多く、短い場合は10分、長い場合でも40分に収まることが多いです。
特典会は60分前後の場合が多いです。

#出力形式
以下の様なJSON形式で出力してください。
{
    "ライブ名":"TOKYO IDOL FESTIVAL2024"
    ,"会場名":"ZeppDiverCity"
    ,"日付":"2024-08-03"
    ,"タイムテーブル":
    [
        {
            "グループ名":"アイドルグループA"
            ,"ライブステージ":[
            {
                "from":"10:00"
                ,"to":"10:20"
                ,"ステージ名":"なし"
            },
            ]
            ,"特典会":[
            {
                "from":"10:30"
                ,"to":"11:30"
                ,"ブース":"A"
            },
            ]
        },
        {
            "グループ名":"架空のアイドル"
            ,"ライブステージ":[
            {
                "from":"10:25"
                ,"to":"10:45"
                ,"ステージ名":"なし"
            },
            ]
            ,"特典会":[
            {
                "from":"10:55"
                ,"to":"11:55"
                ,"ブース":"B"
            },
            ]
        }
    ]
}
"""

In [18]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [45]:
def timetable_detect(image_path, fes_flag=False, user_prompt = "このライブの情報を教えて"):
    if fes_flag:
        system_prompt = system_prompt_fes
    else:
        system_prompt = system_prompt_taiban
    
    base64_image = encode_image(image_path)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                        },
                    },
                ],
            }
        ],
        response_format={"type": "json_object"},
        max_tokens=4096
    )

    return response

In [37]:
image_path = doc_folder + "timetable_sample/20231210_iCON-DOLL-LOUNGE.jpeg"
response = timetable_detect(image_path)
json.loads(response.choices[0].message.content)

{'ライブ名': 'iCON DOLL LOUNGE 2023〜YEARS END SPECIAL 3MAN〜',
 '会場名': 'なし',
 '日付': '2023-12-10',
 'タイムテーブル': [{'グループ名': 'Mirror,Mirror',
   'ライブステージ': [{'from': '13:00', 'to': '13:35'}],
   '特典会': [{'from': '14:45', 'to': '15:45', 'ブース': 'なし'}]},
  {'グループ名': 'THE ORCHESTRA TOKYO',
   'ライブステージ': [{'from': '13:35', 'to': '14:10'}],
   '特典会': [{'from': '14:45', 'to': '15:45', 'ブース': 'なし'}]},
  {'グループ名': 'Ringwanderung',
   'ライブステージ': [{'from': '14:10', 'to': '14:45'}],
   '特典会': [{'from': '14:45', 'to': '15:45', 'ブース': 'なし'}]}]}

In [32]:
image_path = doc_folder + "timetable_sample/20231209_MUD-PARTY.jpeg"
response = timetable_detect(image_path)
json.loads(response.choices[0].message.content)

{'ライブ名': 'MUD PARTY vol.3',
 '会場名': 'SHIROKANE TAKANAWA SELENE b2',
 '日付': '2023-12-09',
 'タイムテーブル': [{'グループ名': 'HACK',
   'ライブステージ': [{'from': '10:00', 'to': '10:20'}],
   '特典会': [{'from': '10:35', 'to': '11:35', 'ブース': 'A'}]},
  {'グループ名': 'アキストゼネコ',
   'ライブステージ': [{'from': '10:20', 'to': '10:40'}],
   '特典会': [{'from': '10:55', 'to': '11:55', 'ブース': 'B'}]},
  {'グループ名': 'TELLMIT',
   'ライブステージ': [{'from': '10:40', 'to': '11:00'}],
   '特典会': [{'from': '11:15', 'to': '12:15', 'ブース': 'A'}]},
  {'グループ名': 'さよならステイチューン',
   'ライブステージ': [{'from': '11:00', 'to': '11:20'}],
   '特典会': [{'from': '11:35', 'to': '12:35', 'ブース': 'C'}]},
  {'グループ名': 'ベンジャス！',
   'ライブステージ': [{'from': '11:20', 'to': '11:40'}],
   '特典会': [{'from': '11:40', 'to': '12:40', 'ブース': 'B'}]},
  {'グループ名': 'zanka',
   'ライブステージ': [{'from': '11:40', 'to': '12:00'}],
   '特典会': [{'from': '12:15', 'to': '13:15', 'ブース': 'A'}]},
  {'グループ名': 'MEWM',
   'ライブステージ': [{'from': '12:05', 'to': '12:30'}],
   '特典会': [{'from': '12:45', 'to': '13:4

In [33]:
image_path = doc_folder + "timetable_sample/20231226_TOKYO-GIRLS-GIRLS.jpeg"
response = timetable_detect(image_path)
json.loads(response.choices[0].message.content)

{'ライブ名': 'TOKYO GIRLS GIRLS',
 '会場名': '品川インターシティホール',
 '日付': '2023-12-26',
 'タイムテーブル': [{'グループ名': '8月のエヴァラベ',
   'ライブステージ': [{'from': '8:20', 'to': '8:30'}],
   '特典会': [{'from': '8:40', 'to': '9:40', 'ブース': 'A'}]},
  {'グループ名': 'ボンコツコンポ',
   'ライブステージ': [{'from': '8:30', 'to': '8:40'}],
   '特典会': [{'from': '8:50', 'to': '9:50', 'ブース': 'B'}]},
  {'グループ名': 'MiX BeRRY',
   'ライブステージ': [{'from': '8:40', 'to': '8:50'}],
   '特典会': [{'from': '9:00', 'to': '10:00', 'ブース': 'C'}]},
  {'グループ名': 'make mie',
   'ライブステージ': [{'from': '8:50', 'to': '9:00'}],
   '特典会': [{'from': '9:10', 'to': '10:10', 'ブース': 'D'}]},
  {'グループ名': 'RiNCENT#',
   'ライブステージ': [{'from': '9:00', 'to': '9:10'}],
   '特典会': [{'from': '9:20', 'to': '10:20', 'ブース': 'D'}]},
  {'グループ名': '天使にはなれない',
   'ライブステージ': [{'from': '9:10', 'to': '9:20'}],
   '特典会': [{'from': '9:30', 'to': '10:30', 'ブース': 'A'}]},
  {'グループ名': 'アルテミスの翼',
   'ライブステージ': [{'from': '9:20', 'to': '9:35'}],
   '特典会': [{'from': '9:50', 'to': '10:50', 'ブース': 'B'}]},
  {

In [34]:
image_path = doc_folder + "timetable_sample/20231203_TODOROCK-IDOL-FESTIVAL.jpeg"
response = timetable_detect(image_path, True)
json.loads(response.choices[0].message.content)

{'ライブ名': 'TODOROKI!! IDOL FESTIVAL 2023',
 '会場名': 'KAWASAKI TODOROKI ARENA',
 '日付': '2023-12-03',
 'タイムテーブル': [{'グループ名': 'yosugala',
   'ライブステージ': [{'from': '10:00', 'to': '10:20', 'ステージ名': 'アリーナ'}],
   '特典会': [{'from': '10:30', 'to': '11:30', 'ブース': 'B'}]},
  {'グループ名': 'Quubi',
   'ライブステージ': [{'from': '10:40', 'to': '11:00', 'ステージ名': 'アリーナ'}],
   '特典会': [{'from': '11:10', 'to': '12:10', 'ブース': 'B'}]},
  {'グループ名': 'シュレーディンガーの犬',
   'ライブステージ': [{'from': '11:10', 'to': '11:30', 'ステージ名': 'アリーナ'}],
   '特典会': [{'from': '11:35', 'to': '12:35', 'ブース': 'A'}]},
  {'グループ名': 'MIGMA SHELTER',
   'ライブステージ': [{'from': '11:40', 'to': '12:00', 'ステージ名': 'アリーナ'}],
   '特典会': [{'from': '12:10', 'to': '13:10', 'ブース': 'B'}]},
  {'グループ名': 'Kolokol',
   'ライブステージ': [{'from': '12:10', 'to': '12:30', 'ステージ名': 'アリーナ'}],
   '特典会': [{'from': '12:35', 'to': '13:35', 'ブース': 'C'}]},
  {'グループ名': 'Ringwanderung',
   'ライブステージ': [{'from': '12:50', 'to': '13:10', 'ステージ名': 'アリーナ'}],
   '特典会': [{'from': '13:15', 'to': '14:15

In [46]:
image_path = doc_folder + "timetable_sample/20240503_JCIF-LIVE.jpeg"
response = timetable_detect(image_path, True, "このライブの情報を教えて。ただし特典会情報はこの画像にはないのでライブステージのみを出力して")
json.loads(response.choices[0].message.content)

{'ライブ名': 'JAPAN CENTRAL IDOL FESTIVAL 2024',
 '会場名': 'AICHI SKY EXPO',
 '日付': '2024-05-03',
 'タイムテーブル': [{'グループ名': 'THE ENCORE',
   'ライブステージ': [{'from': '09:30', 'to': '09:50', 'ステージ名': 'CENTRAL STAGE'},
    {'from': '17:30', 'to': '17:50', 'ステージ名': 'SKY STAGE'}]},
  {'グループ名': 'アンター・ビースマイナー',
   'ライブステージ': [{'from': '09:50', 'to': '10:10', 'ステージ名': 'CENTRAL STAGE'}]},
  {'グループ名': 'dela',
   'ライブステージ': [{'from': '10:10', 'to': '10:30', 'ステージ名': 'CENTRAL STAGE'},
    {'from': '16:10', 'to': '16:30', 'ステージ名': 'OCEAN STAGE'}]},
  {'グループ名': '感覚ロヂック',
   'ライブステージ': [{'from': '10:50', 'to': '11:10', 'ステージ名': 'CENTRAL STAGE'}]},
  {'グループ名': '虹の飛行少女',
   'ライブステージ': [{'from': '11:10', 'to': '11:30', 'ステージ名': 'CENTRAL STAGE'},
    {'from': '16:30', 'to': '16:50', 'ステージ名': 'CORAL STAGE'}]},
  {'グループ名': 'なないろの雨',
   'ライブステージ': [{'from': '11:30', 'to': '11:50', 'ステージ名': 'CENTRAL STAGE'}]},
  {'グループ名': 'ワンダーウィード 天',
   'ライブステージ': [{'from': '11:50', 'to': '12:10', 'ステージ名': 'CENTRAL STAGE'}]},
  {'グループ

In [40]:
print(response.choices[0].message.content)

{
    "ライブ名":"JAPAN CENTRAL IDOL FESTIVAL 2024",
    "会場名":"AICHI SKY EXPO",
    "日付":"2024-05-03",
    "タイムテーブル":
    [
        {
            "グループ名":"THE ENCORE",
            "ライブステージ":[
                {
                    "from":"09:30",
                    "to":"09:50",
                    "ステージ名":"CENTRAL STAGE"
                }
            ]
        },
        {
            "グループ名":"アンチーテーゼ",
            "ライブステージ":[
                {
                    "from":"09:50",
                    "to":"10:10",
                    "ステージ名":"CENTRAL STAGE"
                }
            ]
        },
        {
            "グループ名":"dela",
            "ライブステージ":[
                {
                    "from":"10:10",
                    "to":"10:30",
                    "ステージ名":"CENTRAL STAGE"
                }
            ]
        },
        {
            "グループ名":"藍那サノ@drop tune",
            "ライブステージ":[
                {
                    "from":"10:30",
                    "to":"10:50",

# グループ名の候補出し

In [9]:
import faiss
import numpy as np

# OpenAI APIを使ってテキストをベクトル化する関数
def get_embedding(text, model="text-embedding-3-small"):
    response = client.embeddings.create(input=text, model=model)
    return response.data[0].embedding

# ベクトル化したいデータ
data = pd.read_csv('../../data/idolname.csv', encoding='utf-8')

# 各行のテキストをベクトル化
data['embedding'] = data['idol_group_name'].apply(lambda x: get_embedding(x))

# ベクトルデータの保存
data.to_csv("../../data/idolname_embedding.csv",index=False)
pd.DataFrame(np.array(data['embedding'].tolist()).astype('float32'),index=data["idol_group_name"]).to_csv("../../data/idolname_embedding_data.csv")

In [70]:
#リストを値に持つカラムがあるDataFrameをcsvに保存した時、それを復元する方法
import ast
data = pd.read_csv("../../data/idolname_embedding.csv")
data['embedding'] = data['embedding'].apply(ast.literal_eval)
display(data)

,idol_group_name,embedding
0,18,"[0.030471624806523323, 0.006533531006425619, 0..."
1,150,"[0.035901907831430435, -0.02838958613574505, 0..."
2,369,"[-0.004741288255900145, 0.027815556153655052, ..."
3,963,"[0.032630566507577896, -0.028512146323919296, ..."
4,969,"[0.025754472240805626, 0.014608949422836304, 0..."
...,...,...
6173,Ｂ少女戦士ゴーファイガー,"[0.04238447919487953, 0.036277513951063156, 0...."
6174,Ｆ-cute♡,"[0.028958620503544807, -0.050658416002988815, ..."
6175,Ｏ₂,"[0.017566528171300888, 0.011338524520397186, -..."
6176,［PUZZLE.］,"[-0.025862406939268112, -0.009841745719313622,..."


In [80]:
data = pd.read_csv("../../data/idolname_embedding_data.csv")
display(data)

,idol_group_name,0,1,2,3,4,5,6,7,8,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,18,0.030472,0.006534,0.011962,0.049267,0.018359,0.035873,-0.023447,0.017841,-0.034645,...,0.017159,0.002293,-0.004320,0.000256,0.022165,-0.040783,0.000115,-0.025111,0.021633,0.042666
1,150,0.035902,-0.028390,0.074949,-0.040823,-0.034097,0.004561,-0.058701,0.004480,0.014828,...,0.023876,-0.017703,-0.020965,-0.007585,0.008779,-0.050140,0.002428,0.012200,0.026788,0.039571
2,369,-0.004741,0.027816,0.052825,0.031300,0.004055,-0.009405,-0.011125,0.011094,-0.004695,...,-0.036419,0.000642,0.009899,0.021031,-0.008904,-0.064821,-0.024146,0.001527,0.012381,0.026613
3,963,0.032631,-0.028512,0.038903,0.004677,-0.006748,-0.017725,-0.012126,0.041849,-0.001599,...,-0.022018,0.003614,-0.013615,0.018248,0.012561,-0.037921,-0.008126,-0.009171,0.013916,0.055852
4,969,0.025754,0.014609,0.033231,0.037503,-0.010604,-0.029172,-0.007430,0.029493,-0.002634,...,-0.029630,0.007324,-0.005393,0.022672,0.013884,-0.031583,-0.002746,-0.006709,0.006145,0.019179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6173,Ｂ少女戦士ゴーファイガー,0.042384,0.036278,0.022178,-0.013607,0.008641,-0.026142,-0.002437,0.017828,-0.026828,...,-0.016960,-0.007323,-0.018085,-0.030685,0.006353,-0.016253,-0.024942,0.022906,0.039920,0.018964
6174,Ｆ-cute♡,0.028959,-0.050658,-0.009534,-0.018377,-0.007048,0.003927,-0.053777,0.045904,0.007201,...,0.000273,-0.030952,-0.001826,-0.030671,0.005176,0.034198,-0.018173,-0.005735,-0.030467,-0.001467
6175,Ｏ₂,0.017567,0.011339,-0.059106,0.023773,-0.042479,0.027460,0.012755,0.005509,-0.028884,...,0.034592,-0.002255,-0.006851,0.010691,-0.010762,0.006363,-0.004904,-0.001756,-0.019787,0.018549
6176,［PUZZLE.］,-0.025862,-0.009842,-0.011892,0.026645,0.011670,0.008688,-0.011602,0.032244,0.003192,...,0.016850,-0.024405,0.012870,0.010166,0.009889,0.022193,0.014894,-0.034861,0.025674,0.021721


In [84]:
data = pd.read_csv("../../data/idolname_embedding_data.csv")

# ベクトルデータベースの作成
embeddings = data.drop("idol_group_name",axis=1).values
d = len(embeddings[0])  # 次元数
index = faiss.IndexFlatL2(d)
index.add(embeddings)

# 類似するデータの検索関数
def find_similar(text, k=3):
    embedding = np.array([get_embedding(text)]).astype('float32')
    distances, indices = index.search(embedding, k)
    return indices[0], distances[0]

In [85]:
# テスト検索
query = "Ringuwanderland"
indices, distances = find_similar(query)

# 結果の表示
print("Query:", query)
print("Similar texts and distances:")
for i, dist in zip(indices, distances):
    print(f"Text: {data.iloc[i]['idol_group_name']}, Distance: {dist}")

Query: Ringuwanderland
Similar texts and distances:
Text: Ringwanderung, Distance: 0.6957418322563171
Text: ワールドエンド・ラヴァー, Distance: 1.1682806015014648
Text: Angel Ring, Distance: 1.1744575500488281
